# Day Five

## Task

We have been [tasked](https://adventofcode.com/2017/day/6) with the reallocation of blocks between memory banks.

Given some initial state, part one is about identifying after how many steps we see a repeated state (not necessarily a repeat of the initial state). Part two is about identifying the length of the cycle over which the state repeats.

The crucial difference between the first and second parts is the need to be able to retrospectively determine when a particular state first appeared. This encourages some interesting data structure choices...

### Part One

First, we define a function that performs the reallocation according to the rules laid out.

In [1]:
import copy

def reallocate(banks):
    banks_copy = copy.copy(banks)
    
    blocks, index = max(banks), banks.index(max(banks))
    banks_copy[index] = 0
    
    while blocks:
        banks_copy[(index + 1) % len(banks)] += 1
        index += 1
        blocks -= 1
        
    return banks_copy

Second, we define a function that determines the number of steps before we see a cycle.

In [2]:
def cycle(banks):
    b = copy.copy(banks)
    seen_states = [b]
    
    while True:
        b = reallocate(b)
        
        if b in seen_states:
            return len(seen_states)
            
        seen_states.append(b)

We validate this on our text example

In [3]:
cycle([0, 2, 7, 0])

5

The answer to part one is then given by

In [4]:
cycle([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

12841

The answer to part one is then given by

### Part Two

Since we store the observed states in a list, we can simply find the index of the first occurrence in order to determine the length of the cycle. That is, we redefine `cycle` as follows

In [5]:
def cycle(banks):
    b = copy.copy(banks)
    seen_states = [b]
    
    while True:
        b = reallocate(b)
        
        if b in seen_states:
            return len(seen_states), len(seen_states) - seen_states.index(b)
            
        seen_states.append(b)

Again, we test this on our example with

In [6]:
cycle([0, 2, 7, 0])

(5, 4)

The answer to part two is then given by

In [7]:
cycle([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

(12841, 8038)

### Optimisation

I mentioned at the start that we can make interesting choices with data structures to affect the performance of our program. First, let's look at the results given by `%prun` for our latest `cycle` function.

```
         274519 function calls in 2.941 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.889    2.889    3.059    3.059 <ipython-input-10-ae6cd06bc826>:1(cycle)
    12841    0.101    0.000    0.167    0.000 <ipython-input-4-d3c1671c05e9>:3(reallocate)
    25682    0.022    0.000    0.022    0.000 {built-in method builtins.max}
    12842    0.017    0.000    0.021    0.000 copy.py:66(copy)
   184626    0.016    0.000    0.016    0.000 {built-in method builtins.len}
    12842    0.006    0.000    0.006    0.000 {method 'index' of 'list' objects}
    12842    0.004    0.000    0.004    0.000 {method 'get' of 'dict' objects}
    12840    0.003    0.000    0.003    0.000 {method 'append' of 'list' objects}
        1    0.001    0.001    3.060    3.060 <string>:1(<module>)
        1    0.000    0.000    3.060    3.060 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
```

The majority of the time is spent in the body of `cycle` (as opposed to in `reallocate` which is what I was expecting). By refactoring `b in seen_states` into a function, it's possible to find that most of the time is spent there. How can we make this faster?

Well, there are two ideas that come to mind. First we'll try storing each state in a format that allows for faster comparison (I'll try `str` and `tuple`). Second, we'll try using a different data structure to store state, a `set`.

In [8]:
def cycle_with_str(banks):
    b = copy.copy(banks)
    seen_states = [','.join([str(x) for x in b])]
    
    while True:
        b = reallocate(b)
        
        if ','.join([str(x) for x in b]) in seen_states:
            return len(seen_states), len(seen_states) - seen_states.index(','.join([str(x) for x in b]))
            
        seen_states.append(','.join([str(x) for x in b]))

In [9]:
cycle_with_str([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

(12841, 8038)

This takes a total of 2.470 seconds to execute, a 16% improvement over 2.941 seconds. Now, let's try with tuples.

In [10]:
def cycle_with_tuple(banks):
    b = copy.copy(banks)
    seen_states = [tuple(b)]
    
    while True:
        b = reallocate(b)
        
        if tuple(b) in seen_states:
            return len(seen_states), len(seen_states) - seen_states.index(tuple(b))
            
        seen_states.append(tuple(b))

In [11]:
cycle_with_tuple([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

(12841, 8038)

This takes a total of 2.982 seconds to execute; essentially the same as when using lists and worse than when using strings. Finally, let's use a set to store state. Note that we'll have to use a different object to store the state's position since we require this for the cycle length. Further, we use the string form of the sequence because we a set requires that its contents be hashable.

In [12]:
def cycle_with_set(banks):
    b = copy.copy(banks)
    seen_states = set(','.join([str(x) for x in b]))
    state_position = {}
    
    while True:
        b = reallocate(b)
                      
        if ','.join([str(x) for x in b]) in seen_states:
            return len(seen_states), len(seen_states) - state_position[','.join([str(x) for x in b])]

        state_position[','.join([str(x) for x in b])] = len(seen_states)
        seen_states.add(','.join([str(x) for x in b]))

In [13]:
cycle_with_set([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

(12848, 8038)

This takes a total of 0.387 seconds to execute, only 13% of the time of the original method. Surprising to me but useful to know!